In [1]:
# Direct lammps script implemetation in pyiron 
from pyiron import Project
import numpy as np
import pandas
from jinja2 import Template
import matplotlib.pyplot as plt 
import scipy.constants as sc
from scipy.integrate import cumtrapz
import os

# Project 
pr_1 = Project("/nfshome/deshmukh/pyiron/projects/NASICON/project/hena/hena_2_1/minimization_small")           #! Path for postprocessing 
pr = Project("/nfshome/deshmukh/pyiron/projects/NASICON/project/hena/hena_2_1/glass_small")   


In [5]:
# Calculate quech rate and time 
rate = range(1, 2, 1)
#rate_1 = range(1, 11, 1)

t = []     # time steps in fs 
for i in rate:
    time = ((4000-300)/i)*1000
    t.append(round(time))

# For different colling rate 

strct_no = [74,2,63]#, 112, 117] #, 140,  91,  94,  31,  82, 120] 
temp = [523, 573, 623, 673, 723, 773]
#for k in strct_no:
# Crystal calculations 
#for k in strct_no:
    #job = pr['hena_1_%s_rate_1_k_ps'%k]
    #print(job.output.energy_pot[-1])
#    job.transfer_from_remote()
#    job.compress()

In [ ]:
for k in strct_no:
    for j,i in zip(temp,range(len(temp))):
        job_mini = pr['hena_1_%s_rate_1_k_ps'%k]               # Bulk Crystal
    
        # Get last structure    
        struct_nvt = job_mini.get_structure(iteration_step=-1) # Last frame from minimization
    
        j = pr.create.job.Vasp("hena_1_glass_small_%s_%sk"%(k,j), delete_existing_job=True)
        j.calc_md(temperature=temp[i], n_ionic_steps=25000, n_print=1, time_step=1.0)     # AIMD simulations        
        j.structure = struct_nvt                               # Intial structure   
        j.set_kpoints(scheme="GC")  
        j.input.kpoints.set_kpoints_file(size_of_mesh=['1 1 1'])
        j.set_exchange_correlation_functional("PBE")           # GGA (PBE) potential, also need to test SCAN, rSCAN, r2SCAN     

        #Select PAW potentials 
        #j.list_potentials()
        j.potential.Si = "Si"                                  # 4 valence electron, EMAX = 245 (eV)
        j.potential.P = "P"                                    # 5 valence electron, EMAX = 255 (eV) 
        j.potential.O = "O"                                    # 6 valence electron, EMAX = 400 (eV)   
        j.potential.Na = "Na"                                  # 7 valence electron with 'p' orbitals, EMAX = 260 (eV) 
        j.potential.Zr = "Zr_sv"                               # 12 valence electron with 's' orbitals, EMAX = 230 (eV)  

        # Edit INCAR file 
        j.input.incar["SMASS"] = 0                             # For noose hover algo     
        j.input.incar["MDALGO"] = 2                            # Nose-Hoover thermostat              
        j.input.incar["PREC"] = "Normal"
        j.input.incar["IBRION"] = 0                            # Conjugate gradient method, det. how the ions are updated and moved (remove if ions are not present)  
        j.input.incar["ISMEAR"] = 0                            # O = Gaussian, -1 = Fermi for AIMD 
        j.input.incar["LREAL"] = "A"                           # Projecion in real or reciprocal space   
        j.input.incar["SIGMA"] = 0.05                          # 0.03 or 0.05 for gaussian (1)  ----- Conv
        j.input.incar["EDIFF"] = 1e-5            #!/bin/bash
#SBATCH -J pure_Ni_sf_path
#SBATCH --mail-type=ALL
#SBATCH --mail-user=nag@mm.tu-darmstadt.de
#SBATCH -e ./err_files/err.%x.%j
#SBATCH -o ./out_files/out.%x.%j
#SBATCH -t 2:00:00
#SBATCH -p devel
#SBATCH -A lowsfehea
#SBATCH --nodes=8
#SBATCH --ntasks-per-node=48
#SBATCH --cpus-per-task=1
#SBATCH --no-requeue
​
# load modules
module purge
#module load Stages/2020
#module load Intel/2021.2.0-GCC-10.3.0
#module load OpenMPI/4.1.1
#module load mpi4py/3.0.3-Python-3.8.5
#module load GCCcore/.10.3.0
#module load Python/3.8.5
#module load SciPy-Stack/2021-Python-3.8.5
​
module load Intel/2021.4.0
module load OpenMPI/4.1.1
module load mpi4py/3.1.3
module load Python/3.9.6
module load SciPy-Stack/2021b
​
# Setting/updating environmental variables
export PYTHONPATH="${PYTHONPATH}:/p/home/jusers/nag1/juwels/Some_useful_codes/useful_common_classes/Atomistic_configurations/3D_systems_of_different_geometries/"
export LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:/p/home/jusers/nag1/juwels/Softwares/LAMMPS_installations/intel_2021_2_0_GCC_10_3_0_openmpi_4_1_1/lammps-29Oct20/build"
export PYTHONPATH="${PYTHONPATH}:/p/home/jusers/nag1/juwels/Softwares/LAMMPS_installations/intel_2021_2_0_GCC_10_3_0_openmpi_4_1_1/lammps-29Oct20/python"
export lammps_exe="/p/home/jusers/nag1/juwels/Softwares/LAMMPS_installations/intel_2021_2_0_GCC_10_3_0_openmpi_4_1_1/lammps-29Oct20/build/lmp_intel_cpu_openmpi"
​
​
# print information
echo "Start Time:"
date
echo "This is Job $SLURM_JOB_ID : $SLURM_JOB_NAME on Project Number $SLURM_JOB_ACCOUNT"
echo "Nodes: $SLURM_JOB_NUM_NODES"
echo "PYTHONPATH: $PYTHONPATH"
echo "LD_LIBRARY_PATH: $LD_LIBRARY_PATH"
​
srun python3 $HOME/Some_useful_codes/Specific_problems/Path_to_stacking_fault_creation/sf_creation_path_in_fcc_using_fix_atoms_in_plane_constraint.py ./pure_Ni_sf_path.config
​
echo "End Time:"
date              # No of K-points treated in parallel, must be divi to IBZKPT  
        j.server.cores = 192
        j.executable.executable_path = '/home/vd80naku/pyiron/resources/vasp/bin/run_vasp_6.1.2_default_mpi_gamma.sh'
        j.server.queue = "vasp_l_l2"
        j.run()

In [ ]:
for k in strct_no:
    for j,i in zip(temp,range(len(temp))):
        job_mini = pr['hena_2_1_struct_mini_%s'%k]               # Bulk Crystal
    
        # Get last structure    
        struct_nvt = job_mini.get_structure(iteration_step=-1) # Last frame from minimization
    
        j = pr.create.job.Vasp("hena_1_crystal_small_%s_%sk"%(k,j), delete_existing_job=True)
        j.calc_md(temperature=temp[i], n_ionic_steps=25000, n_print=1, time_step=1.0)     # AIMD simulations        
        j.structure = struct_nvt                               # Intial structure   
        j.set_kpoints(scheme="GC")  
        j.input.kpoints.set_kpoints_file(size_of_mesh=['1 1 1'])
        j.set_exchange_correlation_functional("PBE")           # GGA (PBE) potential, also need to test SCAN, rSCAN, r2SCAN     

        #Select PAW potentials 
        #j.list_potentials()
        j.potential.Si = "Si"                                  # 4 valence electron, EMAX = 245 (eV)
        j.potential.P = "P"                                    # 5 valence electron, EMAX = 255 (eV) 
        j.potential.O = "O"                                    # 6 valence electron, EMAX = 400 (eV)   
        j.potential.Na = "Na"                                  # 7 valence electron with 'p' orbitals, EMAX = 260 (eV) 
        j.potential.Zr = "Zr_sv"                               # 12 valence electron with 's' orbitals, EMAX = 230 (eV)  

        # Edit INCAR file 
        j.input.incar["SMASS"] = 0                             # For noose hover algo     
        j.input.incar["MDALGO"] = 2                            # Nose-Hoover thermostat              
        j.input.incar["PREC"] = "Normal"
        j.input.incar["IBRION"] = 0                            # Conjugate gradient method, det. how the ions are updated and moved (remove if ions are not present)  
        j.input.incar["ISMEAR"] = 0                            # O = Gaussian, -1 = Fermi for AIMD 
        j.input.incar["LREAL"] = "A"                           # Projecion in real or reciprocal space   
        j.input.incar["SIGMA"] = 0.05                          # 0.03 or 0.05 for gaussian (1)  ----- Conv
        j.input.incar["EDIFF"] = 1e-5                          # Global break condition, accurate cal = 1e-8 (2) -- Conv 
        j.input.incar["ALGO"] = 'Normal'                       # For convergence use Fast else Normal
        # Paralellization 
        j.input.incar["ISYM"] = 0                              # no symmetry
        j.input.incar["MAXMIX"] = 40                           # reuse mixer from one MD step to next
        j.input.incar["NELMIN"] = 4                            # number of electronic steps
        j.input.incar["NCORE"] = 12                            # Core for calculations  
        #j.input.incar["KPAR"] = 12                            # No of K-points treated in parallel, must be divi to IBZKPT  
        j.server.cores = 192
        j.executable.executable_path = '/home/vd80naku/pyiron/resources/vasp/bin/run_vasp_6.1.2_default_mpi_gamma.sh'
        j.server.queue = "vasp_l_l2"
        j.run()